In [105]:
import pandas as pd
import numpy as np
import random
import uuid
import re

# Load base MIMIC-III CSVs (adjust paths as needed)
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
admissions = pd.read_csv('ADMISSIONS.csv')
icd_desc = pd.read_csv('D_ICD_DIAGNOSES.csv')
prescriptions = pd.read_csv('PRESCRIPTIONS.csv')
patients = pd.read_csv('PATIENTS.csv')

# Merge ICD descriptions
diagnoses = diagnoses.merge(icd_desc, on='icd9_code', how='inner')
df = diagnoses.merge(admissions,on='subject_id',how='inner')
# Merge prescriptions with diagnoses on HADM_ID
df = df.merge(prescriptions, on='subject_id', how='inner', suffixes=('_diag', '_rx'))

# Merge with patient demographics
df = df.merge(patients[['subject_id', 'gender']], on='subject_id', how='inner')
# df.info()
# Filter out nulls
df = df.dropna(subset=['drug'])
# Generate UUID mapping for anonymization
# STEP 1: Generate patient-level features per SUBJECT_ID
# ------------------------------------------------
unique_subjects = df[['subject_id', 'gender']].drop_duplicates().copy()
unique_subjects['PatientID'] = [str(uuid.uuid4()) for _ in range(len(unique_subjects))]
unique_subjects['Age'] = np.random.randint(18, 90, size=len(unique_subjects))
unique_subjects['Weight_kg'] = np.round(np.random.normal(70, 15, len(unique_subjects)), 1)
unique_subjects['Height_cm'] = np.round(np.random.normal(170, 10, len(unique_subjects)), 1)
unique_subjects['BMI'] = np.round(unique_subjects['Weight_kg'] / ((unique_subjects['Height_cm'] / 100) ** 2), 1)


# Simulate/enrich additional columns
num_rows = df.shape[0]


def random_multi_choice(pool, max_items=2):
    k = random.randint(1, max_items)
    return ', '.join(random.sample(pool, k))
#generate only none values for allergy without additional values from pool
def generate_drug_allergy(pool):
    """If 'None' is selected, return only 'None'. Otherwise, return a combo without 'None'."""
    if random.random() < 0.4:  # Adjust probability of having no allergies (e.g., 40%)
        return 'None'
    else:
        non_none_allergies = [a for a in pool if a != 'None']
        k = random.randint(1, min(2, len(non_none_allergies)))
        return ', '.join(random.sample(non_none_allergies, k))  
        

# def clean_med_name(med):
#     # Remove content in parentheses, doses, and units
#     med = re.sub(r'\(.*?\)', '', med)  # Remove anything in ()
#     med = re.sub(r'\b\d+.*?\b', '', med)  # Remove numbers and units (e.g., 500mg, 100/50)
#     med = re.sub(r'\b(mg|ml|g|mcg|mEq|units|%)\b', '', med, flags=re.IGNORECASE)
#     med = re.sub(r'[^a-zA-Z\s/-]', '', med)  # Keep only letters, space, /, -
#     med = med.strip()
#     return med



# def clean_med_list(meds):
#     if isinstance(meds, list):
#         return [str(m).strip() for m in meds if str(m).strip()]
#     elif isinstance(meds, str):
#         # Remove punctuation and split the string
#         cleaned = re.sub(r'[^\w\s]', '', meds)
#         return [m.strip() for m in cleaned.split() if m.strip()]
#     else:
#         return []


def clean_med_list_to_string(meds):
    remove_terms = {
        'mg', 'ml', 'units', 'sulfate', 'chloride', 'flush', 'g', 'mcg', 'hr',
        'meq', 'wcodeine', 'd5w', 'ns', 'cv1', 'cvl', 'sodium', 'potassium',
        'senna', 'thiamine', 'tuberculin', 'pneumococcal', 'glucose', 'citrate', 'unitsml'
    }

    tokens = []

    if isinstance(meds, str):
        # Remove brackets and punctuation, then split
        meds_cleaned = re.sub(r"[^\w\s]", "", meds)
        tokens = meds_cleaned.split()

    elif isinstance(meds, list):
        # Flatten if nested
        flat = []
        for item in meds:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        # Convert to string tokens
        tokens = [str(t) for t in flat]

    else:
        return ''

    cleaned = []
    for token in tokens:
        token_clean = token.strip().lower()
        if not token_clean or token_clean.isnumeric() or token_clean in remove_terms:
            continue
        cleaned.append(token_clean.capitalize())

    # Remove duplicates, preserve order
    return ', '.join(dict.fromkeys(cleaned))


   
# chronic_conditions_pool = ['Hypertension', 'Diabetes', 'Asthma','Sepsis','Gout NOS',]
allergy_pool = ['Penicillin', 'Aspirin','Shellfish', 'None']


chronic_map = (
    diagnoses[['subject_id', 'icd9_code']]
    .merge(icd_desc[['icd9_code', 'short_title']], on='icd9_code', how='left')
    .dropna(subset=['short_title'])
    .drop_duplicates()
    .groupby('subject_id')['short_title']
    .apply(lambda x: ', '.join(x.unique()[:3]))  # Limit to top 3 unique diagnoses
    .reset_index(name='Chronic_Conditions')
)

# Step 2: Merge into the unique_subjects DataFrame (one row per patient)
unique_subjects = unique_subjects.merge(chronic_map, on='subject_id', how='left')

# Fill missing with 'None' if any
unique_subjects['Chronic_Conditions'] = unique_subjects['Chronic_Conditions'].fillna('None')
# unique_subjects['Drug_Allergies'] = [random_multi_choice(allergy_pool) for _ in range(len(unique_subjects))]
unique_subjects['Drug_Allergies'] = [generate_drug_allergy(allergy_pool) for _ in range(len(unique_subjects))]


# ------------------------------------------------
# STEP 2: Merge synthetic patient data back into full DataFrame
# ------------------------------------------------
df = df.merge(unique_subjects, on=['subject_id', 'gender'], how='left')


effectiveness_levels = ['High', 'Moderate', 'Low']
adverse_reactions_pool = ['None', 'Nausea', 'Rash', 'Fatigue']
durations = ['5 days', '7 days', '10 days', '14 days']
dosages = ['5mg', '10mg', '20mg', '50mg']

# Apply synthetic logic
df['Chronic_Conditions'] = df['diagnosis'].fillna('Unknown')
df['Recommended_Medication'] = df['drug'].apply(clean_med_list_to_string)
df['NDC'] = df['ndc'].fillna(0).astype(int)
df['Symptoms'] = df['long_title'].fillna('Unknown')
df['Dosage'] = df['prod_strength']
df['Duration'] = [random.choice(durations) for _ in range(num_rows)]
df['Treatment_Effectiveness'] = [random.choices(effectiveness_levels, weights=[0.6, 0.3, 0.1])[0] for _ in range(num_rows)]
df['Adverse_Reactions'] = [random.choices(adverse_reactions_pool, weights=[0.5, 0.2, 0.2, 0.1])[0] for _ in range(num_rows)]
df['Recovery_Time_Days'] = df['Treatment_Effectiveness'].map({'High': 5, 'Moderate': 10, 'Low': 15})

# Select and rename final columns
final_df = df[[
    'subject_id', 'Age', 'gender', 'Weight_kg', 'Height_cm', 'BMI',
    'Chronic_Conditions', 'Drug_Allergies', 
    'icd9_code', 'diagnosis', 'Symptoms','NDC',
    'Recommended_Medication', 'Dosage', 'Duration',
    'Treatment_Effectiveness', 'Adverse_Reactions', 'Recovery_Time_Days'
]].rename(columns={
    'subject_id': 'PatientID',
    'gender': 'Gender',
    'icd9_code': 'Diagnosis',
    'diagnosis': 'Diagnosis_Description'
})
final_df = final_df.dropna(subset=[
    'Recommended_Medication', 'Symptoms', 'Diagnosis'
])
# final_df['NDC'] = final_df['NDC'].astype(int)
final_df = final_df[final_df['NDC'] != 0]

# final_df['Medication_Dosage'] = final_df['Recommended_Medication'] + ' (' + final_df['Dosage'].astype(str) + ')'
final_df['Recommended_Medication_tuple'] = final_df['Recommended_Medication'].apply(tuple)


df_unique_rx = final_df.drop_duplicates(subset=[
    'PatientID', 'Diagnosis', 'Recommended_Medication_tuple'
])
# df_unique_rx
df_final = df_unique_rx.groupby('PatientID').agg({
    'Age': 'first',
    'Gender': 'first',
    'BMI': 'first',
    'Weight_kg':'first',
    'Height_cm':'first',
    'BMI':'first',
    'Chronic_Conditions':'first',
    'Symptoms':'first',
    'Diagnosis': 'first',
    'Recommended_Medication': 'first',
    'NDC':'first',
    'Dosage':'first',
    'Duration':'first',
    'Treatment_Effectiveness':'first', 
    'Adverse_Reactions':'first',
    'Recovery_Time_Days':'first'
    
}).reset_index()


print(df_final.head())


   PatientID  Age Gender   BMI  Weight_kg  Height_cm   Chronic_Conditions  \
0      10006   37      F  25.5       71.8      167.8               SEPSIS   
1      10013   58      F  23.1       82.2      188.8               SEPSIS   
2      10017   76      F  19.8       48.4      156.4     HUMERAL FRACTURE   
3      10019   51      M  29.9       84.0      167.7  ALCOHOLIC HEPATITIS   
4      10026   22      F  20.4       51.8      159.4           STROKE/TIA   

                                      Symptoms Diagnosis  \
0                                       Sepsis     99591   
1                       Unspecified septicemia      0389   
2  Closed fracture of surgical neck of humerus     81201   
3                       Unspecified septicemia      0389   
4                     Intracerebral hemorrhage       431   

  Recommended_Medication          NDC             Dosage Duration  \
0              Glipizide  51079081120           10MG TAB   5 days   
1             Furosemide     74610204 

In [ ]:
df_final.shape

In [ ]:
df_final.info()

In [111]:
import pandas as pd
import numpy as np
import uuid
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
patient_id_counter = 10000
# --- 1. Diagnosis, Symptom, Medication Mapping ---
diagnosis_data = {
    '250.00': {
        'description': 'Diabetes mellitus without mention of complication',
        'symptoms': ['Increased thirst', 'Frequent urination', 'Fatigue'],
        'medications': [('Metformin', '500mg', '000931048')]
    },
    '401.9': {
        'description': 'Essential hypertension',
        'symptoms': ['Headache', 'Dizziness', 'Blurred vision'],
        'medications': [('Lisinopril', '10mg', '435470282')]
    },
    '493.90': {
        'description': 'Asthma, unspecified type, unspecified',
        'symptoms': ['Wheezing', 'Shortness of breath', 'Chest tightness'],
        'medications': [('Albuterol', '90mcg', '000748038')]
    },
    '780.6': {
        'description': 'Fever',
        'symptoms': ['Chills', 'Sweating', 'High temperature'],
        'medications': [('Acetaminophen', '500mg', '005910465')]
    },
    '724.2': {
        'description': 'Lumbago (lower back pain)',
        'symptoms': ['Back pain', 'Limited movement', 'Stiffness'],
        'medications': [('Ibuprofen', '200mg', '005271402')]
    },
    '530.81': {
        'description': 'Gastroesophageal reflux disease (GERD)',
        'symptoms': ['Heartburn', 'Regurgitation', 'Chest discomfort'],
        'medications': [('Omeprazole', '20mg', '001730834')]
    },
    '486': {
        'description': 'Pneumonia, organism unspecified',
        'symptoms': ['Cough', 'Fever', 'Shortness of breath'],
        'medications': [('Azithromycin', '250mg', '167140492')]
    },
    '599.0': {
        'description': 'Urinary tract infection, site not specified',
        'symptoms': ['Burning urination', 'Frequent urination', 'Pelvic pain'],
        'medications': [('Nitrofurantoin', '100mg', '000931713')]
    },
    '784.0': {
        'description': 'Headache',
        'symptoms': ['Throbbing pain', 'Sensitivity to light', 'Nausea'],
        'medications': [('Sumatriptan', '50mg', '000245910')]
    },
    '787.01': {
        'description': 'Nausea with vomiting',
        'symptoms': ['Nausea', 'Vomiting', 'Abdominal discomfort'],
        'medications': [('Ondansetron', '4mg', '001730601')]
    },
    '311': {
        'description': 'Depressive disorder, not elsewhere classified',
        'symptoms': ['Persistent sadness', 'Fatigue', 'Sleep disturbance'],
        'medications': [('Fluoxetine', '20mg', '000494900')]
    },
    '285.9': {
        'description': 'Anemia, unspecified',
        'symptoms': ['Fatigue', 'Pale skin', 'Dizziness'],
        'medications': [('Ferrous sulfate', '325mg', '001210649')]
    },
    '729.1': {
        'description': 'Myalgia and myositis, unspecified',
        'symptoms': ['Muscle pain', 'Stiffness', 'Tenderness'],
        'medications': [('Cyclobenzaprine', '10mg', '000939752')]
    },
    '780.52': {
        'description': 'Insomnia, unspecified',
        'symptoms': ['Difficulty falling asleep', 'Restlessness', 'Fatigue'],
        'medications': [('Zolpidem', '10mg', '002282550')]
    },
    '787.91': {
        'description': 'Diarrhea',
        'symptoms': ['Loose stools', 'Abdominal cramps', 'Urgency to defecate'],
        'medications': [('Loperamide', '2mg', '005740397')]
    }

}


# --- 2. Utility Functions ---
def generate_drug_allergy(pool):
    if random.random() < 0.4:  # 40% chance of 'None'
        return 'None'
    else:
        non_none_allergies = [a for a in pool if a != 'None']
        k = random.randint(1, min(2, len(non_none_allergies)))
        return ', '.join(random.sample(non_none_allergies, k))

def compute_bmi(weight, height):
    return round(weight / ((height / 100) ** 2), 1)



def generate_synthetic_patient_row():
    global patient_id_counter
    diagnosis_code = random.choice(list(diagnosis_data.keys()))
    data = diagnosis_data[diagnosis_code]

    symptoms = random.sample(data['symptoms'], k=random.randint(1, len(data['symptoms'])))
    med, dosage, ndc = random.choice(data['medications'])

    weight = round(np.random.normal(70, 15), 1)
    height = round(np.random.normal(170, 10), 1)

    patient_row = {
        'PatientID': patient_id_counter ,
        'Age': np.random.randint(18, 90),
        'Gender': random.choice(['M', 'F']),
        'Weight_kg': weight,
        'Height_cm': height,
        'BMI': compute_bmi(weight, height),
        'Chronic_Conditions': data['description'].split(',')[0],
        'Drug_Allergies': generate_drug_allergy(['Penicillin', 'Aspirin', 'Shellfish', 'None']),
        'Diagnosis': diagnosis_code,
        'Diagnosis_Description': data['description'],
        'Symptoms': ', '.join(symptoms),
        'Recommended_Medication': med,
        'Dosage': dosage,
        'NDC': ndc,
        'Duration': random.choice(['5 days', '7 days', '10 days']),
        'Treatment_Effectiveness': random.choices(['High', 'Moderate', 'Low'], weights=[0.6, 0.3, 0.1])[0],
        'Adverse_Reactions': random.choices(['None', 'Nausea', 'Rash', 'Fatigue'], weights=[0.5, 0.2, 0.2, 0.1])[0],
    }
    patient_id_counter += 10
    return patient_row

# --- 3. Generate Dataset ---
synthetic_data = []
for _ in range(50_000):
    row = generate_synthetic_patient_row()
    row['Recovery_Time_Days'] = {'High': 5, 'Moderate': 10, 'Low': 15}[row['Treatment_Effectiveness']]
    synthetic_data.append(row)

# --- 4. Create DataFrame ---
syn_df = pd.DataFrame(synthetic_data)
# syn_df['Medication_Dosage'] = syn_df['Recommended_Medication'] + ' (' + syn_df['Dosage'] + ')'

df_synfinal = syn_df.groupby('PatientID').agg({
    'Age': 'first',
    'Gender': 'first',
    'BMI': 'first',
    'Weight_kg':'first',
    'Height_cm':'first',
    'BMI':'first',
    'Chronic_Conditions':'first',
    'Symptoms':lambda x:list(x),
    'Diagnosis': 'first',
    'Recommended_Medication': 'first',
    'NDC':'first',
    'Dosage':'first',
    'Duration':'first',
    'Treatment_Effectiveness':'first', 
    'Adverse_Reactions':'first',
    'Recovery_Time_Days':'first'
    
}).reset_index()
# Preview the data
print(df_synfinal.head())
print("\nTotal rows generated:", len(syn_df))


   PatientID  Age Gender   BMI  Weight_kg  Height_cm   Chronic_Conditions  \
0      10000   89      M  27.3       77.5      168.6  Depressive disorder   
1      10010   41      M  17.8       53.3      173.2             Headache   
2      10020   47      M  21.3       62.8      171.6             Headache   
3      10030   38      M  22.6       61.3      164.7                Fever   
4      10040   66      M  26.7       91.9      185.4               Anemia   

                     Symptoms Diagnosis Recommended_Medication        NDC  \
0        [Persistent sadness]       311             Fluoxetine  000494900   
1                    [Nausea]     784.0            Sumatriptan  000245910   
2                    [Nausea]     784.0            Sumatriptan  000245910   
3  [Chills, High temperature]     780.6          Acetaminophen  005910465   
4        [Dizziness, Fatigue]     285.9        Ferrous sulfate  001210649   

  Dosage Duration Treatment_Effectiveness Adverse_Reactions  \
0   20mg  1

In [ ]:
df_synfinal['Dosage'].value_counts()

In [113]:
final_dataset  = pd.concat([df_final, df_synfinal], ignore_index=True)
print(final_dataset)

       PatientID  Age Gender   BMI  Weight_kg  Height_cm  \
0          10006   37      F  25.5       71.8      167.8   
1          10013   58      F  23.1       82.2      188.8   
2          10017   76      F  19.8       48.4      156.4   
3          10019   51      M  29.9       84.0      167.7   
4          10026   22      F  20.4       51.8      159.4   
...          ...  ...    ...   ...        ...        ...   
50089     509950   23      F  28.9       73.6      159.7   
50090     509960   88      M  29.8       75.6      159.3   
50091     509970   20      M  24.5       74.9      174.7   
50092     509980   43      M  15.2       37.5      157.2   
50093     509990   69      M  19.8       71.8      190.4   

              Chronic_Conditions  \
0                         SEPSIS   
1                         SEPSIS   
2               HUMERAL FRACTURE   
3            ALCOHOLIC HEPATITIS   
4                     STROKE/TIA   
...                          ...   
50089        Depressive dis

In [115]:
final_dataset.to_csv('patient_icd_rx_grouped.csv', index=False)

In [ ]:
final_dataset.shape

In [118]:
final_dataset

,PatientID,Age,Gender,BMI,Weight_kg,Height_cm,Chronic_Conditions,Symptoms,Diagnosis,Recommended_Medication,NDC,Dosage,Duration,Treatment_Effectiveness,Adverse_Reactions,Recovery_Time_Days
0,10006,37,F,25.5,71.8,167.8,SEPSIS,Sepsis,99591,Glipizide,51079081120,10MG TAB,5 days,High,Nausea,5
1,10013,58,F,23.1,82.2,188.8,SEPSIS,Unspecified septicemia,0389,Furosemide,74610204,40MG/4ML VIAL,14 days,High,Fatigue,5
2,10017,76,F,19.8,48.4,156.4,HUMERAL FRACTURE,Closed fracture of surgical neck of humerus,81201,Hydromorphone,74131230,2mg/mL Syringe,5 days,Moderate,Fatigue,10
3,10019,51,M,29.9,84.0,167.7,ALCOHOLIC HEPATITIS,Unspecified septicemia,0389,Propofol,310030011,1000mg/100mL Vial,10 days,Moderate,None,10
4,10026,22,F,20.4,51.8,159.4,STROKE/TIA,Intracerebral hemorrhage,431,Metoprolol,55390007310,5mg/5mL Vial,7 days,Moderate,Nausea,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50089,509950,23,F,28.9,73.6,159.7,Depressive disorder,"[Fatigue, Sleep disturbance]",311,Fluoxetine,000494900,20mg,7 days,High,Nausea,5
50090,509960,88,M,29.8,75.6,159.3,Nausea with vomiting,"[Vomiting, Abdominal discomfort]",787.01,Ondansetron,001730601,4mg,10 days,High,Nausea,5
50091,509970,20,M,24.5,74.9,174.7,Lumbago (lower back pain),"[Back pain, Limited movement]",724.2,Ibuprofen,005271402,200mg,7 days,Moderate,Nausea,10
50092,509980,43,M,15.2,37.5,157.2,Headache,"[Sensitivity to light, Nausea, Throbbing pain]",784.0,Sumatriptan,000245910,50mg,10 days,High,None,5
